In [65]:
%pip install pandas
%pip install numpy
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [66]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [67]:
# Step 2: Load Data
train_df = pd.read_csv('train (1).csv')
test_df = pd.read_csv('test (1).csv')
combine = [train_df, test_df]

In [68]:
# Step 3: Begin Data Preprocessing

# Drop non-essential columns
train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

In [69]:
# Feature Engineering: Extract Title from Name
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
C:\Users\aakas\AppData\Local\Temp\ipykernel_4968\519829561.py:3: SyntaxWarning: invalid escape sequence '\.'
  dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)


In [70]:
# Consolidate and replace rare titles
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [71]:
# Convert Title to a numerical feature
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

In [72]:
# Drop original 'Name' column and 'PassengerId' from training set
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]

In [73]:
# Convert 'Sex' to a numerical feature
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

In [74]:
# Impute missing 'Age' values based on Pclass and Sex
guess_ages = np.zeros((2,3))
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & (dataset['Pclass'] == j+1)]['Age'].dropna()
            age_guess = guess_df.median()
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1), 'Age'] = guess_ages[i,j]
    dataset['Age'] = dataset['Age'].astype(int)

In [75]:
# Bin 'Age' into ordinal categories (Corrected from original notebook)
for dataset in combine:
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4

In [76]:
# Feature Engineering: Create 'FamilySize' and 'IsAlone'
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

In [77]:
# Drop original family-related columns
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

In [78]:
# Feature Engineering: Create 'Age*Class'
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

In [79]:
# Impute missing 'Embarked' values with the most frequent port
freq_port = train_df.Embarked.dropna().mode()[0]
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)

In [80]:
# Convert 'Embarked' to a numerical feature
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [81]:
# Impute missing 'Fare' in test set with the median
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)

C:\Users\aakas\AppData\Local\Temp\ipykernel_4968\2836775784.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)


In [82]:
# Bin 'Fare' into ordinal categories
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

In [83]:
# Step 4: Prepare Data for Modeling
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()

In [84]:
# Step 5: Train and Evaluate Models

# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred_log = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)

In [85]:
# Support Vector Machines
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred_svc = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)

In [86]:
# K-Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred_knn = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)

In [87]:
# Gaussian Naive Bayes
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred_gaussian = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)

In [88]:
# Perceptron
perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred_perceptron = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)

In [89]:
# Linear SVC
linear_svc = LinearSVC(dual=False) # dual=False to avoid ConvergenceWarning
linear_svc.fit(X_train, Y_train)
Y_pred_linear_svc = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)

In [90]:
# Stochastic Gradient Descent
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred_sgd = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)

In [91]:
# Decision Tree
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred_dt = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)

In [92]:
# Random Forest
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred_rf = random_forest.predict(X_test)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)

In [93]:
# Step 6: Consolidate Model Scores
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
print("--- Model Accuracy on Training Data ---")
print(models.sort_values(by='Score', ascending=False))

--- Model Accuracy on Training Data ---
                        Model  Score
3               Random Forest  86.64
8               Decision Tree  86.64
0     Support Vector Machines  82.83
1                         KNN  82.83
2         Logistic Regression  81.37
7                  Linear SVC  79.46
5                  Perceptron  79.35
4                 Naive Bayes  76.88
6  Stochastic Gradient Decent  74.97


In [98]:


# --- Step 1: Load Data ---
try:
    train_df = pd.read_csv('train (1).csv')
    test_df = pd.read_csv('test (1).csv')
    
    # Store test PassengerId for submission file
    test_passenger_id = test_df['PassengerId']
    
    # Combine train and test sets for consistent processing
    combined_df = pd.concat([train_df.drop('Survived', axis=1), test_df], ignore_index=True)
    
    # --- Step 2: Feature Engineering & Imputation ---
    
    
    # Create 'Title' feature
    combined_df['Title'] = combined_df.Name.str.extract(' ([A-Za-z]+)\\.', expand=False)
    combined_df['Title'] = combined_df['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    combined_df['Title'] = combined_df['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})
    
    # Create 'IsAlone' feature
    combined_df['FamilySize'] = combined_df['SibSp'] + combined_df['Parch'] + 1
    combined_df['IsAlone'] = 0
    combined_df.loc[combined_df['FamilySize'] == 1, 'IsAlone'] = 1
    combined_df.drop(columns=['SibSp', 'Parch', 'FamilySize'], inplace=True)
    
    # Impute missing values
    combined_df['Embarked'].fillna(combined_df['Embarked'].mode()[0], inplace=True)
    combined_df['Fare'].fillna(combined_df['Fare'].median(), inplace=True)
    combined_df['Age'] = combined_df.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))

    # --- Step 3: One-Hot Encode Categorical Features (Improvement) ---
    categorical_features = ['Embarked', 'Title', 'Sex']
    combined_df = pd.get_dummies(combined_df, columns=categorical_features, drop_first=True)

    # Drop columns that are not useful
    combined_df.drop(columns=['Ticket', 'Cabin', 'Name', 'PassengerId'], inplace=True)
    
    # --- Step 4: Split back into training and testing sets ---
    X_train = combined_df.iloc[:len(train_df)]
    X_test = combined_df.iloc[len(train_df):]
    Y_train = train_df['Survived']

    # --- Step 5: Scale Numerical Features (Improvement) ---
    scaler = StandardScaler()
    numerical_features = ['Age', 'Fare', 'Pclass']
    X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
    X_test[numerical_features] = scaler.transform(X_test[numerical_features])
    
    # Save the processed files
    X_train.to_csv("improved_train.csv", index=False)
    X_test.to_csv("improved_test.csv", index=False)
    
    # --- Step 6: Train and Evaluate Models on Improved Data ---
    models_data = []
    
    # Define models
    models_to_run = {
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Support Vector Machines": SVC(),
        "KNN": KNeighborsClassifier(n_neighbors=3),
        "Naive Bayes": GaussianNB(),
        "Perceptron": Perceptron(),
        "Linear SVC": LinearSVC(dual=False, max_iter=2000),
        "Stochastic Gradient Decent": SGDClassifier(),
        "Decision Tree": DecisionTreeClassifier(),
        "Random Forest": RandomForestClassifier(n_estimators=100)
    }

    # Train and score models
    for name, model in models_to_run.items():
        model.fit(X_train, Y_train)
        score = round(model.score(X_train, Y_train) * 100, 2)
        models_data.append({'Model': name, 'Score': score})
    
    # --- Step 7: Display Improved Accuracies ---
    improved_models_df = pd.DataFrame(models_data)
    print("--- Improved Model Accuracies on Training Data ---")
    print(improved_models_df.sort_values(by='Score', ascending=False))

except FileNotFoundError:
    print("Execution halted. Please upload 'train (1).csv' and 'test (1).csv' to proceed.")

--- Improved Model Accuracies on Training Data ---
                        Model  Score
7               Decision Tree  98.43
8               Random Forest  98.43
2                         KNN  87.54
1     Support Vector Machines  83.16
0         Logistic Regression  82.15
5                  Linear SVC  81.37
6  Stochastic Gradient Decent  79.91
3                 Naive Bayes  78.11
4                  Perceptron  76.09


C:\Users\aakas\AppData\Local\Temp\ipykernel_4968\1686104532.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['Embarked'].fillna(combined_df['Embarked'].mode()[0], inplace=True)
C:\Users\aakas\AppData\Local\Temp\ipykernel_4968\1686104532.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alw